# Boundary 

The task is to combine current velocities from an oceanographic model without tides with tidal current.

* The oceanographic model data is vertically resolved and available in a vertical transect as a Dfs2 with daily timestep
* The tidal model is vertically integrated and available in a line transect as a Dfs1 with hourly timestep
* The spatial grid is identical

## Steps
1. Interpolate in time
2. Calculate $\mathbf{U}_{combined} = \mathbf{U}_{tide} + \mathbf{U}_{residual}$
3. Write to new dfs2

In [34]:
import numpy as np
import matplotlib.pyplot as plt
from mikeio import Dfs2, Dfs1

dfs2 = Dfs2(r"../tests/testdata/uv_vertical_daily.dfs2")
ds_surge = dfs2.read()
ds_surge

<mikeio.DataSet>
Dimensions: (6, 100, 40)
Time: 2017-01-01 00:00:00 - 2017-01-06 00:00:00
Items:
  0:  U-velocity <u velocity component> (meter per sec)
  1:  V-velocity <v velocity component> (meter per sec)

In [35]:
dfs1 = Dfs1(r"../tests/testdata/vu_tide_hourly.dfs1")
ds_tide = dfs1.read()
ds_tide

<mikeio.DataSet>
Dimensions: (145, 40)
Time: 2017-01-01 00:00:00 - 2017-01-07 00:00:00
Items:
  0:  Tidal current component (geographic North) <v velocity component> (meter per sec)
  1:  Tidal current component (geographic East) <u velocity component> (meter per sec)

### 1. Interpolate in time

In [36]:
ds_surge_h = ds_surge.interp_time(ds_tide.time)
ds_surge_h

<mikeio.DataSet>
Dimensions: (145, 100, 40)
Time: 2017-01-01 00:00:00 - 2017-01-07 00:00:00
Items:
  0:  U-velocity <u velocity component> (meter per sec)
  1:  V-velocity <v velocity component> (meter per sec)

### 2. Combine

Note that the naming and order is inconsistent between the two data sources

In [37]:
u_tide = ds_tide["Tidal current component (geographic East)"]
u_tide = np.expand_dims(u_tide, 1)
u_tide.shape

(145, 1, 40)

In [38]:
u_surge = ds_surge_h["U-velocity"]
u_surge.shape

(145, 100, 40)

In [39]:
u_combined = u_tide + u_surge
u_combined.shape

(145, 100, 40)

In [40]:
v_tide = ds_tide["Tidal current component (geographic North)"]
v_tide = np.expand_dims(v_tide, 1)
v_surge = ds_surge_h["V-velocity"]
v_combined = v_tide + v_surge

### 3. Write to dfs2

In [41]:
from mikeio import Dataset
ds_combined = Dataset(data=[u_combined,v_combined],time=ds_tide.time,items=ds_surge.items)
ds_combined

<mikeio.DataSet>
Dimensions: (145, 100, 40)
Time: 2017-01-01 00:00:00 - 2017-01-07 00:00:00
Items:
  0:  U-velocity <u velocity component> (meter per sec)
  1:  V-velocity <v velocity component> (meter per sec)

In [42]:
ds_combined = ds_combined.dropna()
ds_combined

<mikeio.DataSet>
Dimensions: (121, 100, 40)
Time: 2017-01-01 00:00:00 - 2017-01-06 00:00:00
Items:
  0:  U-velocity <u velocity component> (meter per sec)
  1:  V-velocity <v velocity component> (meter per sec)

In [43]:
dfsnew = Dfs2()

dfsnew.write("uv_combined.dfs2", ds_combined)

# Clean up


In [44]:
import os
os.remove("uv_combined.dfs2")